In [67]:
import json
import requests
import xml.etree.ElementTree as ET
import os
from tqdm import tqdm
import pickle
import re
import time

clarinpl_url = "http://ws.clarin-pl.eu/nlprest2/base"
user_mail = "demo2019@nlpday.pl"

In [68]:
gram_mapper = {
'subst':  'rzecz',
'depr':   'rzecz',
'num':    'rzecz',
'numcol': 'rzecz',
'adj':    'przym',
'adja':   'przym',
'adjp':   'przym',
'adjc':   'przym',
'adv':    'przys',
'ppron12':'rzecz',
'ppron3': 'rzecz',
'siebie': 'rzecz',
'fin':    'czas',
'bedzie': 'czas',
'aglt':   'czas',
'praet':  'czas',
'impt':   'czas',
'imps':   'czas',
'inf':    'czas',
'pcon':   'czas',
'pant':   'czas',
'ger':    'czas',
'pact':   'czas',
'ppas':   'czas',
'winien': '?',
'pred':   '?',
'prep':   '?',
'conj':   '?',
'comp':   '?',
'qub':    '?',
'brev':   '?',
'burk':   '?',
'interj': '?',
'interp': '?',
'xxx':    '?',
'ign':    '?',
}

url = clarinpl_url + "/process"
lpmn = "wcrft2"
headers = {'Content-type': 'text/plain; charset=utf-8', "Accept-Language": "pl"}
wordsDf = pd.DataFrame(columns=["base", "parts_of_speech"])
wiki_train_dir = "./dataset/Zad4/wiki_train_34_categories_data/"
xmls_train_dir = "./zad4_xmls/"

In [79]:
for fileName in tqdm(os.listdir(wiki_train_dir)):
    with open(wiki_train_dir+fileName, "r") as file:
        text = file.read()
    payload = {'text': text, 'lpmn': lpmn, 'user': user_mail}
    r = requests.post(url, 
        data=json.dumps(payload),
        headers=headers)
    ccl = r.content.decode('utf-8')
    with open(xmls_train_dir+fileName[:-4]+".xml", "w") as file:
        file.write(ccl)

  1%|          | 82/6890 [1:29:45<124:11:31, 65.67s/it] 


KeyboardInterrupt: 

In [ ]:
clarinpl_url = "http://ws.clarin-pl.eu/nlprest2/base"
url = clarinpl_url + "/process"
lpmn = "wcrft2"
headers = {'Content-type': 'text/plain; charset=utf-8', "Accept-Language": "pl"}
user_mail = "demo2019@nlpday.pl"
wiki_test_dir = "./dataset/Zad4/wiki_test_34_categories_data/"
save_test_dir = "./zad4_test_xmls/"
for fileName in tqdm(os.listdir(wiki_test_dir)):
    with open(wiki_test_dir+fileName, "r") as file:
        text = file.read()
    payload = {'text': text, 'lpmn': lpmn, 'user': user_mail}
    r = requests.post(url,
        data=json.dumps(payload),
        headers=headers)
    ccl = r.content.decode('utf-8')
    with open(save_test_dir+fileName[:-4]+".xml", "w") as file:
        file.write(ccl)

In [ ]:
def getMostFrequent(wordsDf, part_of_speach, vec_size):
    for train_xml in xmls_train_dir:
    category = re.findall(r'[a-zA-Z\-]+', fileName)[0]
    addToWordsDf(ccl, wordsDf, category)
    wordsDf.to_csv('wordsDf.csv')
    
    gram_mapper[wordfreq]
    
def addToWordFreq(xml_string, wordfreq):
    tree = ET.ElementTree(ET.fromstring(xml_string))
    root = tree.getroot()
    for tok in root.iter('tok'):
        for lex in tok.iter('lex'):
            if "disamb" in lex.attrib and lex.attrib['disamb']=="1":
                base = lex.find('base').text
                if base not in wordfreq.keys():
                    wordfreq[base] = 1
                else:
                    wordfreq[base] += 1

In [57]:
def addToWordsDf(xml_path, wordsDf, category):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    newWords = pd.DataFrame(columns=["base", "parts_of_speech", "n_occurrences"])
    for tok in root.iter('tok'):
        for lex in tok.iter('lex'):
            if "disamb" in lex.attrib and lex.attrib['disamb']=="1":
                base = lex.find('base').text
                ctag = lex.find('ctag').text
                partOfSpeach = gram_mapper[ctag.split(":")[0]]
                if wordsDf[wordsDf['base'] == base].empty:
                    record = pd.DataFrame(data={"base": [base], "parts_of_speech": [partOfSpeach], "n_occurrences": 1})
                    wordsDf = pd.concat([wordsDf, record], ignore_index=True)
                else:
                    wordsDf[wordsDf['base'] == base, ["n_occurrences"]].iloc[0] += 1